In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import os

from torch.utils.data import WeightedRandomSampler
from torchvision import datasets, transforms
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import StepLR

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# Fixed seeds
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
seed_everything(SEED)

# Model Training

In [4]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #output = output.max(1, keepdim=True)[1]
        output = nn.LogSoftmax(dim=1)(output)
        #print("Target:",target)
        #print("output:",[np.argmax(i) for i in output.cpu().detach().numpy()])
        #print("Output:",output)
        #print("-"*40)        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            output = nn.LogSoftmax(dim=1)(output)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            #print("Test Predicted:",pred)

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
################### Parameters for training ###################
batch_size = 30
epochs = 50
log_interval = 10
save_model = True
lr = 0.01
gamma = 0.7
no_of_classes = 5


################### Class weightage ###################
class_info  = pd.read_csv("./data/train/separated/train.csv")
class_count = class_info['labels'].value_counts().values.tolist()
target_list = class_info['labels'].values.tolist()

target_list = torch.tensor(target_list)
target_list = target_list[torch.randperm(len(target_list))]

class_weights = 1./torch.tensor(class_count, dtype=torch.float)
class_weights_all = class_weights[target_list]
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

################### Directory for Train and Val ###################
traindir = "./data/train/separated/train"
valdir = "./data/train/separated/val"

################### Defining GPU ###################
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

################### Physical parameters for training ###################
kwargs = {'batch_size': batch_size}
if use_cuda:
    kwargs.update({'num_workers': 1,
                   'pin_memory': True},
                 )

################### Defining transformations for image and dataset object ###################
transform=transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.ImageFolder(traindir,transform=transform)
dataset2 = datasets.ImageFolder(valdir,transform=transform)

################### Defining train and val dataloader ###################
train_loader = torch.utils.data.DataLoader(dataset1, sampler=weighted_sampler, **kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

################### Defining modified network ###################
model = EfficientNet.from_name('efficientnet-b1')
for param in model.parameters():
    param.requires_grad = True

num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, no_of_classes)
model = model.to(device)

################### Criterion, Optimizer and Scheduler ###################
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

################### Main ###################
for epoch in range(1, epochs + 1):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

################### Save model ###################
if save_model:
    torch.save(model.state_dict(), "aptos_blindness_effnet_b1_weighted.pt")

Train Epoch: 1 [0/2346 (0%)]	Loss: 1.716060
Train Epoch: 1 [300/2346 (13%)]	Loss: 1.290683
Train Epoch: 1 [600/2346 (25%)]	Loss: 1.230907
Train Epoch: 1 [900/2346 (38%)]	Loss: 1.135546
Train Epoch: 1 [1200/2346 (51%)]	Loss: 0.832001
Train Epoch: 1 [1500/2346 (63%)]	Loss: 0.951843
Train Epoch: 1 [1800/2346 (76%)]	Loss: 0.763330
Train Epoch: 1 [2100/2346 (89%)]	Loss: 1.236535

Test set: Average loss: 1.8460, Accuracy: 200/733 (27%)

Train Epoch: 2 [0/2346 (0%)]	Loss: 1.051167
Train Epoch: 2 [300/2346 (13%)]	Loss: 1.206290
Train Epoch: 2 [600/2346 (25%)]	Loss: 0.916297
Train Epoch: 2 [900/2346 (38%)]	Loss: 0.881769
Train Epoch: 2 [1200/2346 (51%)]	Loss: 1.130997
Train Epoch: 2 [1500/2346 (63%)]	Loss: 0.729409
Train Epoch: 2 [1800/2346 (76%)]	Loss: 0.735041
Train Epoch: 2 [2100/2346 (89%)]	Loss: 0.911981

Test set: Average loss: 1.9742, Accuracy: 200/733 (27%)

Train Epoch: 3 [0/2346 (0%)]	Loss: 0.773660
Train Epoch: 3 [300/2346 (13%)]	Loss: 1.412137
Train Epoch: 3 [600/2346 (25%)]	Loss: 0

Train Epoch: 19 [2100/2346 (89%)]	Loss: 0.630119

Test set: Average loss: 0.7225, Accuracy: 530/733 (72%)

Train Epoch: 20 [0/2346 (0%)]	Loss: 0.683705
Train Epoch: 20 [300/2346 (13%)]	Loss: 0.348976
Train Epoch: 20 [600/2346 (25%)]	Loss: 0.525582
Train Epoch: 20 [900/2346 (38%)]	Loss: 0.557600
Train Epoch: 20 [1200/2346 (51%)]	Loss: 0.522482
Train Epoch: 20 [1500/2346 (63%)]	Loss: 0.652910
Train Epoch: 20 [1800/2346 (76%)]	Loss: 0.656141
Train Epoch: 20 [2100/2346 (89%)]	Loss: 0.801524

Test set: Average loss: 0.7225, Accuracy: 529/733 (72%)

Train Epoch: 21 [0/2346 (0%)]	Loss: 0.474922
Train Epoch: 21 [300/2346 (13%)]	Loss: 0.782797
Train Epoch: 21 [600/2346 (25%)]	Loss: 0.476713
Train Epoch: 21 [900/2346 (38%)]	Loss: 0.708365
Train Epoch: 21 [1200/2346 (51%)]	Loss: 0.484219
Train Epoch: 21 [1500/2346 (63%)]	Loss: 0.447614
Train Epoch: 21 [1800/2346 (76%)]	Loss: 0.583475
Train Epoch: 21 [2100/2346 (89%)]	Loss: 0.587450

Test set: Average loss: 0.7222, Accuracy: 529/733 (72%)

Train E

Train Epoch: 38 [900/2346 (38%)]	Loss: 0.539022
Train Epoch: 38 [1200/2346 (51%)]	Loss: 0.631324
Train Epoch: 38 [1500/2346 (63%)]	Loss: 0.523476
Train Epoch: 38 [1800/2346 (76%)]	Loss: 0.609610
Train Epoch: 38 [2100/2346 (89%)]	Loss: 0.540422

Test set: Average loss: 0.7213, Accuracy: 529/733 (72%)

Train Epoch: 39 [0/2346 (0%)]	Loss: 0.630865
Train Epoch: 39 [300/2346 (13%)]	Loss: 0.336051
Train Epoch: 39 [600/2346 (25%)]	Loss: 0.534247
Train Epoch: 39 [900/2346 (38%)]	Loss: 0.574930
Train Epoch: 39 [1200/2346 (51%)]	Loss: 0.661987
Train Epoch: 39 [1500/2346 (63%)]	Loss: 0.810320
Train Epoch: 39 [1800/2346 (76%)]	Loss: 0.671461
Train Epoch: 39 [2100/2346 (89%)]	Loss: 0.663562

Test set: Average loss: 0.7215, Accuracy: 529/733 (72%)

Train Epoch: 40 [0/2346 (0%)]	Loss: 0.785158
Train Epoch: 40 [300/2346 (13%)]	Loss: 0.725972
Train Epoch: 40 [600/2346 (25%)]	Loss: 0.878055
Train Epoch: 40 [900/2346 (38%)]	Loss: 0.691657
Train Epoch: 40 [1200/2346 (51%)]	Loss: 0.788594
Train Epoch: 40 [

# Testing

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score

In [7]:
model_path = "model_1.pt"
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = EfficientNet.from_name('efficientnet-b1').to(device)
model.load_state_dict(torch.load(model_path))

Loaded pretrained weights for efficientnet-b1


<All keys matched successfully>

In [8]:
testdir = "./data/train/separated/test"
transform=transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

dataset1 = datasets.ImageFolder(testdir,transform=transform)
test_loader = torch.utils.data.DataLoader(dataset1)

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
test_loss = 0

true_labels = []
preds = []

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        true_labels.append(labels)
        output = model(images)
        output = nn.LogSoftmax(dim=1)(output)
        test_loss += F.nll_loss(output, labels, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        preds.append(pred)
        correct += pred.eq(labels.view_as(pred)).sum().item()
        
        total += labels.size(0)

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the test images: 71.8696397941681 %


In [9]:
preds = [i.cpu().detach().numpy()[0][0] for i in preds]
true_labels = [i.cpu().detach().numpy()[0] for i in true_labels]

In [10]:
conf = confusion_matrix(true_labels, preds)
print(conf)
print("Accuracy score : "+str(accuracy_score(true_labels, preds)))
print("kappa score : "+str(cohen_kappa_score(true_labels, preds)))

[[273   1  14   0   0]
 [  3   5  51   0   0]
 [ 13   5 141   0   0]
 [  0   0  30   0   0]
 [  2   3  42   0   0]]
Accuracy score : 0.7186963979416809
kappa score : 0.5469763519116051
